# ***Introduction***



In this report, I will be looking at Covid-19 data for the United States. The data is purposfully cut down from all 50 states for the sake of saving space, but the actions I perfrom on it will hold up if applied in a distributed computing setting where I can use the resources of mulitple nodes. I will narrow my original dataset down to the three [most republican states](https://worldpopulationreview.com/state-rankings/most-republican-states) which are Wyoming, Utah, and Oklahoma. Then I will take the corespoinding [most democratic states](https://worldpopulationreview.com/state-rankings/most-democratic-statesps://) which are Hawaii, Vermont, and California. 


I chose this project because during the Covid-19 pandemic, there was a lot of debate between political parties regarding what policies (if any) we should put in place to fight the virus. By looking at the most extreme ends of the political spectrum, hopfully we can uncover details about what policies worked best to fight the virus, in hopes of being more prepaired for the next global pandemic.

# Setup / Imports / SparkContext

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from time import sleep
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import types as sparktypes
from pyspark.sql.functions import col

sc = SparkContext() 
spark = SparkSession(sc)

sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 52 kB/s 
     |████████████████████████████████| 199 kB 92.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b8f12e7ab628be47f2a10c8fe144af1f9d53779633d2dbaa4b09624e1a861a21
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


# ***Datasets***


For this final project, I obtained United States Covid-19 data from datahub.io. I used the [us_simplified.csv](https://datahub.io/core/covid-19#resource-us_simplified) dataset because of its straightforward nature and clean headings. 

Structure:
The us_simplified.csv file has 6 columns: 


*   Date
*   Admin2 - The county of the given state.

*   Province/State
*   Confimed - The number of confimed cases for a given date.


*   Deaths - The number of deaths for a given date.

*   Country/Region


The time period of when this data is collected is 2020-01-22 to 2022-01-23. As alluded to in the introduction, I will be focusing on data for Wyoming, Utah, Oklahoma, Hawaii, Vermont, and California.

# Import Data

In [ ]:
from google.colab import drive # this is how I access my data from the drive.
drive.mount('/content/drive')

Mounted at /content/drive


Turn the csv file into a dataframe:

In [ ]:
original_df = spark.read.option("header",True).csv("/content/drive/MyDrive/CSC 369/us_simplified.csv")
original_df.show()

+----------+-------+--------------+---------+------+--------------+
|      Date| Admin2|Province/State|Confirmed|Deaths|Country/Region|
+----------+-------+--------------+---------+------+--------------+
|2020-01-22|Autauga|       Alabama|        0|     0|            US|
|2020-01-23|Autauga|       Alabama|        0|     0|            US|
|2020-01-24|Autauga|       Alabama|        0|     0|            US|
|2020-01-25|Autauga|       Alabama|        0|     0|            US|
|2020-01-26|Autauga|       Alabama|        0|     0|            US|
|2020-01-27|Autauga|       Alabama|        0|     0|            US|
|2020-01-28|Autauga|       Alabama|        0|     0|            US|
|2020-01-29|Autauga|       Alabama|        0|     0|            US|
|2020-01-30|Autauga|       Alabama|        0|     0|            US|
|2020-01-31|Autauga|       Alabama|        0|     0|            US|
|2020-02-01|Autauga|       Alabama|        0|     0|            US|
|2020-02-02|Autauga|       Alabama|        0|   

## ***Question***
When deciding on what question to ask, I focused on one that would lead us to discover common factors that minimized the impact of Covid-19 most effictivly, while transforming the data in meaningful ways.


*   Does political party of a given state have an association with the ratio of confirmed cases to deaths?

  *   What other factors might be influencing this ratio?
  *   How can we use the results of this analysis to make better decisions during the next pandmeic?








# ***Problem Decompostition***


 In the following section you will see me manipulate the data in many ways en route to our final result. Here is a breif overview of these steps:


*   Take the full dataset of 50 states and narrow it down to the six I want.
*   Make sure that each column in my dataframe is the correct data type for easier manipulation down the road.

*   Find the sum of all confimed cases and deaths for each state over our time period.
*   Use the sums found in the previous step to compute the ratio of confimed cases to deaths.

*   Find the average ratio for each political party.








#Narrow down the data to my desired 6 states 

In [ ]:
df = original_df.withColumnRenamed("Province/State", "State").withColumnRenamed("Country/Region", "Country") #rename for clarity
df = df.select("Date", "State", "Confirmed", "Deaths", "Country") # select only the columns we need.
df = df.filter((df["State"] == "Wyoming") | 
               (df["State"] == "Utah") |
               (df["State"] == "Oklahoma") |
               (df["State"] == "Hawaii") |
               (df["State"] == "Vermont") |
               (df["State"] == "California")) # now we have only the 6 states that we need.
#cast so that we can do math later on
df = df.withColumn("Confirmed", col("Confirmed").cast("int")).withColumn("Deaths", col("Deaths").cast("int"))
df.show()

+----------+----------+---------+------+-------+
|      Date|     State|Confirmed|Deaths|Country|
+----------+----------+---------+------+-------+
|2020-01-22|California|        0|     0|     US|
|2020-01-23|California|        0|     0|     US|
|2020-01-24|California|        0|     0|     US|
|2020-01-25|California|        0|     0|     US|
|2020-01-26|California|        0|     0|     US|
|2020-01-27|California|        0|     0|     US|
|2020-01-28|California|        0|     0|     US|
|2020-01-29|California|        0|     0|     US|
|2020-01-30|California|        0|     0|     US|
|2020-01-31|California|        0|     0|     US|
|2020-02-01|California|        0|     0|     US|
|2020-02-02|California|        0|     0|     US|
|2020-02-03|California|        0|     0|     US|
|2020-02-04|California|        0|     0|     US|
|2020-02-05|California|        0|     0|     US|
|2020-02-06|California|        0|     0|     US|
|2020-02-07|California|        0|     0|     US|
|2020-02-08|Californ

Now we find the total deaths and total confirmed cases for each state.

In [ ]:
df1 = df.groupBy("State").sum("Confirmed")
df2 = df.groupBy("State").sum("Deaths")
df3 = df1.join(df2, "State")
df3.show()

+----------+--------------+-----------+
|     State|sum(Confirmed)|sum(Deaths)|
+----------+--------------+-----------+
|    Hawaii|      24421302|     276712|
|California|    1839595402|   27149306|
|      Utah|     199344756|    1112698|
|   Vermont|      12554607|     128605|
|  Oklahoma|     223813728|    3244662|
|   Wyoming|      31941561|     373257|
+----------+--------------+-----------+



All that we have left now is to find the ratio of confirmed cases to deaths for each state.


In [ ]:
df_ratio = df3.withColumn("Ratio", df3["sum(Confirmed)"]/df3["sum(Deaths)"])

Add an additional column so that we can clearly see the political party of the given state.
R = Republican 
D = Democrat

In [ ]:
from pyspark.sql.functions import when
df_ratio = df_ratio.withColumn("Party", \
                   when((df["State"] == "Wyoming"), "R") \
                   .when((df["State"] == "Utah"), "R") \
                   .when((df["State"] == "Oklahoma"), "R") \
                   .when((df["State"] == "Hawaii"), "D") \
                   .when((df["State"] == "Vermont"), "D") \
                   .when((df["State"] == "California"), "D"))

# ***Results***
The chart below shows the ratio for each of our six states.

In [ ]:
df_ratio.orderBy("Ratio", ascending=False).show()

+----------+--------------+-----------+-----------------+-----+
|     State|sum(Confirmed)|sum(Deaths)|            Ratio|Party|
+----------+--------------+-----------+-----------------+-----+
|      Utah|     199344756|    1112698|179.1544120686835|    R|
|   Vermont|      12554607|     128605|97.62145328719723|    D|
|    Hawaii|      24421302|     276712|  88.255305154818|    D|
|   Wyoming|      31941561|     373257|85.57524976088861|    R|
|  Oklahoma|     223813728|    3244662|68.97905791111678|    R|
|California|    1839595402|   27149306|67.75846874317892|    D|
+----------+--------------+-----------+-----------------+-----+





It is important to understand what our data means before we extrapolate from it. In this context, the larger the ratio of confirmed cases to deaths, the better. In other words, you could say that for every 179 people who got Covid-19 in Utah from 2020-01-22 to 2022-01-23, one person died. 

We can see that the state with the worst ratio is California, and the one with the best is Utah. Right off the bat, this points to a stark difference between Republican and Democratic states and their coresponding ratios. With that being said, we must be careful in only looking at the minimum and maximum ratio, as we don't want outliers to skew our view of what is actually happening.

To solve this problem, we sould look at the average ratio for each political party:

In [ ]:
df_ratio.groupBy("Party").avg("Ratio").show()


+-----+------------------+
|Party|        avg(Ratio)|
+-----+------------------+
|    D| 84.54507572839806|
|    R|111.23623991356295|
+-----+------------------+



Now, we see a different perspective as the average ratio for the dems is significantly smaller than that of the conservative states. 

# So what is this data telling us?
In short, not very much. There are so many factors that could contribute to the difference in the ratio between states. Below I will list a few that came to mind.


*   Quality of health care within the state.
*   Percentage of the population that is elderly.

*   Availablilty of the vaccine.
*   Percentage of individuals that took the vaccine.

*   Amount of people in close-quarters living situations
*   Overall wealth of the individuals within the population of the state.

Although it is very convienient to try to say one political party handled the pandemic better than the other, there are so many factors that could have influenced the ratio of cases to deaths that it would be irresponsible to comment on policy recommendations. 

One concrete thing that we can do with these results would be to look at the way California handled the pandemic verses Utah and see if we can learn any lessons for next time. Unfortuantly that research is beyond the scope of this project, so I will save that for another time.




# ***Conclusion***
It was very exciting processing this data and telling a story using these distributed computing tools. Knowing how important data is in our modern world makes me even more motivated to explore this topic in greater detail. The most important thing I learned is that we must be careful drawing conclusions from a data analysis because we often fail to consider all factors at play. 